In [1]:
from getpass import getpass

import psycopg2
from psycopg2.extras import execute_values

# 1. Connecting to a Database

In [2]:
%%bash
# Create a database to work in
createdb psycopg2_test

In [3]:
# Establish a connection to the database
connection = psycopg2.connect(
    dbname="psycopg2_test",
    password=getpass("User password: "),
    host="localhost",
    port="5432",
)

User password:  ········


# 2. Executing Queries

In [4]:
with connection:
    with connection.cursor() as cursor:
        # Print postgresql version
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()[0]
        print(f"Server version:\n\n{db_version}")

Server version:

PostgreSQL 15.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 13.2.1 20230728 (Red Hat 13.2.1-1), 64-bit


## 2.1 Creating tables

In [5]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            CREATE TABLE utf8_chars (
                code_point   integer PRIMARY KEY,
                character    text NOT NULL
            );
            """
        )

        cursor.execute(
            """
            CREATE TABLE exports (
                type        text NOT NULL,
                commodity   text NOT NULL,
                mass_in_kg  numeric(11, 2) NOT NULL,
                price       numeric(15, 2) NOT NULL,
                date        timestamp NOT NULL
                CHECK (mass_in_kg > 0 AND price > 0)
            );
            """
        )

## 2.2 Inserting data

In [6]:
with connection:
    with connection.cursor() as cursor:
        # Insert one row into the "utf8_chars" table
        cursor.execute("INSERT INTO utf8_chars VALUES (%s, %s)", (1, chr(1)))

        # Insert multiple rows (very inefficient)
        cursor.executemany(
            "INSERT INTO utf8_chars VALUES (%s, %s)",
            [(code_point, chr(code_point)) for code_point in range(2, 50)],
        )
        # Insert multiple rows efficiently
        execute_values(
            cur=cursor,
            sql="INSERT INTO utf8_chars VALUES %s",
            argslist=[(code_point, chr(code_point)) for code_point in range(51, 128)],
        )

        # Import data from a CSV file into the "exports" table
        cursor.execute("SET datestyle='MDY';")  # Set date format
        with open("exports_data.csv") as exports_file:
            header = exports_file.readline()
            cursor.copy_from(file=exports_file, table="exports", sep=",", null="")

## 2.3 Fetching data

In [7]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM utf8_chars;")
        print(
            "fetchone(): ",
            cursor.fetchone(),  # Fetch the next row of a query result set
            "\n\nfetchmany(5): ",
            cursor.fetchmany(5),  # Fetch the next `size=5` rows of a query result
            "\n\nfetchall(): ",
            cursor.fetchall(),  # Fetch all rows of a query result not yet returned
        )

        cursor.execute(
            """
            SELECT type, commodity, price
              FROM exports
              WHERE (price > %s)
              ORDER BY price DESC;
            """,
            ((1_000_000_000),),
        )
        print("\n\nTransactions worth billions: \n", *cursor.fetchall(), sep="\n")

fetchone():  (1, '\x01') 

fetchmany(5):  [(2, '\x02'), (3, '\x03'), (4, '\x04'), (5, '\x05'), (6, '\x06')] 

fetchall():  [(7, '\x07'), (8, '\x08'), (9, '\t'), (10, '\n'), (11, '\x0b'), (12, '\x0c'), (13, '\r'), (14, '\x0e'), (15, '\x0f'), (16, '\x10'), (17, '\x11'), (18, '\x12'), (19, '\x13'), (20, '\x14'), (21, '\x15'), (22, '\x16'), (23, '\x17'), (24, '\x18'), (25, '\x19'), (26, '\x1a'), (27, '\x1b'), (28, '\x1c'), (29, '\x1d'), (30, '\x1e'), (31, '\x1f'), (32, ' '), (33, '!'), (34, '"'), (35, '#'), (36, '$'), (37, '%'), (38, '&'), (39, "'"), (40, '('), (41, ')'), (42, '*'), (43, '+'), (44, ','), (45, '-'), (46, '.'), (47, '/'), (48, '0'), (49, '1'), (51, '3'), (52, '4'), (53, '5'), (54, '6'), (55, '7'), (56, '8'), (57, '9'), (58, ':'), (59, ';'), (60, '<'), (61, '='), (62, '>'), (63, '?'), (64, '@'), (65, 'A'), (66, 'B'), (67, 'C'), (68, 'D'), (69, 'E'), (70, 'F'), (71, 'G'), (72, 'H'), (73, 'I'), (74, 'J'), (75, 'K'), (76, 'L'), (77, 'M'), (78, 'N'), (79, 'O'), (80, 'P'), (81, 'Q

## 2.4 Deleting data

In [8]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            DELETE FROM utf8_chars
              WHERE code_point BETWEEN %(lower)s AND %(upper)s
              RETURNING *;
            """,
            {"lower": 65, "upper": 75}
        )
        print("Deleted rows: ", *cursor.fetchall())

Deleted rows:  (65, 'A') (66, 'B') (67, 'C') (68, 'D') (69, 'E') (70, 'F') (71, 'G') (72, 'H') (73, 'I') (74, 'J') (75, 'K')


## 2.5 Aggregations

In [9]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            SELECT commodity, sum(mass_in_kg) AS total_mass
              FROM exports
              GROUP BY commodity
              ORDER BY total_mass DESC
              LIMIT 5;
            """
        )
        print("Most traded commodities by mass: \n", *cursor.fetchall(), sep="\n")

Most traded commodities by mass: 

('Roses', Decimal('105164149.83'))
('Avocado', Decimal('31225859.63'))
('Mixed vegetables', Decimal('28129937.97'))
('Fine beans', Decimal('23090599.07'))
('Mango', Decimal('14047648.26'))


In [10]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            SELECT type, commodity, round(avg(price / mass_in_kg), 2) AS price_per_kg
              FROM exports
              GROUP BY type, commodity
              ORDER BY price_per_kg DESC
              LIMIT 5;
            """
        )
        print("Top 5 exports by average price-per-kg: \n", *cursor.fetchall(), sep="\n")

Top 5 exports by average price-per-kg: 

('Cut-flowers', 'Geranium cuttings', Decimal('230466.16'))
('Cut-flowers', 'Asters', Decimal('34444.89'))
('Vegetables', 'Tomato', Decimal('26495.82'))
('Cut-flowers', 'Cuttings', Decimal('7687.88'))
('Cut-flowers', 'Phlox', Decimal('7069.64'))


In [11]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            SELECT type, max(mass_in_kg), min(mass_in_kg), round(avg(mass_in_kg), 2)
              FROM exports
              GROUP BY type
              ORDER BY max DESC;
            """
        )
        print(
            "Maximum, minimum and average mass by type: \n", *cursor.fetchall(), sep='\n'
        )

Maximum, minimum and average mass by type: 

('Cut-flowers', Decimal('11049185.48'), Decimal('0.47'), Decimal('164424.40'))
('Fruits', Decimal('5302741.82'), Decimal('2.00'), Decimal('416633.76'))
('Vegetables', Decimal('3565356.17'), Decimal('1.09'), Decimal('144837.01'))


# 3. Conclusion & Clean Up

In [12]:
with connection:
    with connection.cursor() as cur:
        # Delete a table
        cur.execute("DROP TABLE utf8_chars;")


# Close the connection to database
connection.close()

In [13]:
%%bash
# Delete the "psycopg2_test" database
dropdb psycopg2_test